In [ ]:
!pip install datasets transformers evaluate seqeval pynvml pyyaml h5py huggingface_hub

In [ ]:
import pandas as pd
import os
import requests
import time
import sys
import json
from requests_oauthlib import OAuth1Session
from requests_oauthlib import OAuth1
import base64
from collections import Counter, defaultdict
import transformers
import random
import datasets
import tokenize
import io
import re
import time
import math
import datetime

# Data Collect

In [ ]:
# GitHub Access Token
with open('github_token', 'r') as accestoken:
    access = accestoken.readline().replace("\n","")

In [ ]:
%run -i commits_from_GitHub.py

In [ ]:
%run -i commit_diffs.py

In [ ]:
%run -i diff_commit_data.py

# Data Preprocessing

In [ ]:
%run -i clean_and_shorten_data.py

In [ ]:
#roberta and codebert tokenization and embedding - input which one after file name
%run -i create_dataset_roberta.py clean_dataset_short.pickle codebert

In [ ]:
#distilbert
%run -i create_dataset_distilbert.py clean_dataset_short.pickle distilbert

# Model Development

In [ ]:
!pip install datasets transformers evaluate seqeval pynvml pyyaml h5py huggingface_hub

In [ ]:
import pandas as pd
import os
import json
import random
import datasets
import pickle
from datasets import load_metric
import evaluate
import numpy as np
from pynvml import *
import tensorflow as tf

import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
#tqdmn = tqdm.notebook.tqdm

import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import AutoModelForTokenClassification, AutoTokenizer

import sklearn
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForTokenClassification,
   AdamW,
   AdamWeightDecay)

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import load_dataset

In [ ]:
#file = 'roberta_tagsandtokens_PADDED.pickle'
file = 'distilbert_tagsandtokens_PADDED.pickle'
#file = 'codebert_tagsandtokens_PADDED.pickle'
with open(file, 'rb') as data:
    dataset = pickle.load(data)
dataset

In [ ]:
model_choice = "roberta-base"
tok_choice = "roberta-base"
#model_choice = 'microsoft/codebert-base'
#tok_choice = 'microsoft/codebert-base'
#model_choice = "distilbert-base-uncased"
#tok_choice = "distilbert-base-uncased"

In [ ]:
label_list = ['O',      #0
              'B-rce',  #1
              'I-rce',  #2
              'B-oob',  #3
              'I-oob',  #4
              'B-xss',  #5
              'I-xss',  #6
              'B-sql',  #7
              'I-sql',  #8
              'B-iiv',  #9
              'I-iiv',  #10
              'B-pat',  #11
              'I-pat',  #12
             ]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

In [ ]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_choice, num_labels = len(label_list))

model.config.id2label = id2label
model.config.label2id = label2id

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
optimizer = optim.AdamW(params=model.parameters(), lr=2e-5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
from torch.utils.data import DataLoader

train_data = DataLoader(
    dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_data = DataLoader(
    dataset["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
from transformers import get_scheduler

num_epochs = 10
num_training_steps = num_epochs * len(train_data)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
train_loss = []
model.train()
for epoch in tqdmn(range(num_epochs)):
    current_loss = 0
    for i, batch in enumerate(tqdmn(train_data)):
        # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        outputs = model(**batch)
        loss = outputs.loss #outputs[0]
        loss.backward()
        current_loss += loss.item()
        optimizer.step()
        lr_scheduler.step()

        optimizer.zero_grad()
        train_loss.append(current_loss / 10)
        current_loss = 0

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import login

notebook_login()

In [ ]:
model.save_pretrained('PyNERModel_dev')

In [ ]:
from huggingface_hub import whoami
whoami()

In [ ]:
model.push_to_hub(repo_id = '') #need to input user id

# Model Test

In [ ]:
!pip install huggingface_hub datasets transformers torchview

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import login

notebook_login()

In [ ]:
from transformers import AutoTokenizer
#tok_choice = 'roberta-base'
tok_choice = 'microsoft/codebert-base'
#tok_choice = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tok_choice)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('mmeberg/CoCo_PyVulDet_NER')
#'mmeberg/CoCo_PyVulDet_NER'
#'mmeberg/RoCo_PyVulDet_NER'
#'mmeberg/RoRo_PyVulDet_NER'
#'mmeberg/CoRo_PyVulDet_NER'
#'mmeberg/DiDi_PyVulDet_NER'

In [ ]:
import tqdm
tqdmn = tqdm.notebook.tqdm

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
#model.config

## GitHub Test Set

In [ ]:
import pickle
import datasets

file = 'PADDED_distilbert.pickle'
with open(file, 'rb') as data:
    dataset = pickle.load(data)
dataset

In [ ]:
import pandas as pd
test_df = pd.DataFrame(dataset['test'])
test_df = test_df[['input_ids', 'attention_mask', 'labels']]
test_df.head(2)

In [ ]:
test_data = datasets.Dataset.from_pandas(test_df)
test_data.set_format(type='torch')

In [ ]:
test_data.format['type']

In [ ]:
from torch.utils.data import DataLoader

test_data = DataLoader(
    test_data, shuffle=True, batch_size=8, collate_fn=data_collator
)

In [ ]:
label_list = ['O',      #0
              'B-rce',  #1
              'I-rce',  #2
              'B-oob',  #3
              'I-oob',  #4
              'B-xss',  #5
              'I-xss',  #6
              'B-sql',  #7
              'I-sql',  #8
              'B-iiv',  #9
              'I-iiv',  #10
              'B-pat',  #11
              'I-pat',  #12
             ]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

In [ ]:
#Expand results so no longer lists of lists
for_cm_true = [y for x in all_true for y in x]
print(len(for_cm_true))
for_cm_pred = [y for x in all_pred for y in x]
print(len(for_cm_pred))

In [ ]:
labels_name = list(label2id.keys())
print(labels_name)
labels_id = list(id2label.keys())
print(labels_id)

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
import sklearn.metrics as skm
import seaborn as sns

In [ ]:
labels_id_11 = list(id2label.keys())[1:]
print(labels_id_11)
labels_name_11 = list(label2id.keys())[1:]
print(labels_name_11)

In [ ]:
cm = confusion_matrix(for_cm_true, for_cm_pred, labels =labels_id_11)

In [ ]:
# Normalize
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=labels_name_11, yticklabels=labels_name_11)
plt.ylabel('Actual')
plt.xlabel('Predicted')
#plt.savefig('plot4.jpg', format='jpg', dpi=1200)
plt.show(block=False)

In [ ]:
import numpy as np
confusion_np = cmn#.np()
FP = confusion_np.sum(axis=0) - np.diag(confusion_np)
FN = confusion_np.sum(axis=1) - np.diag(confusion_np)
TP = np.diag(confusion_np)
TN = confusion_np.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP)
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
#F1-Score
F1 = TP/(TP+.5*(FP+FN))

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

metrics = {'FP':FP, 'FN':FN, 'TP':TP, 'TN':TN, 'TPR':TPR, 'TNR':TNR, 'PPV':PPV, 'NPV':NPV, 'FPR':FPR, 'FNR':FNR, 'FDR':FDR, 'F1':F1, 'ACC':ACC}
for k, v in metrics.items():
    print(k)
    for x in zip(labels_name_11, list(v)):
        print(x[0], x[1])
    print()

In [ ]:
test_tns = [
 'rce', #1
 'oob', #2
 'xss', #3
 'sql', #4
 'iiv', #5
 'pat'  #6
 ]

test_tns_ids = [ 1, 2, 3, 4, 5, 6]

cwes = [
 'CWE-94', #1
 'CWE-787', #2
 'CWE-79', #3
 'CWE-89', #4
 'CWE-20', #5
 'CWE-22'  #6
 ]


id2label_test = {i: label for i, label in enumerate(test_tns,1)}
label2id_test = {label: i for i, label in enumerate(test_tns,1)}

all_true_test = [label2id_test[id2label[x].split('-')[1]] if x!=0 else 0 for x in for_cm_true]
all_pred_test = [label2id_test[id2label[x].split('-')[1]] if x!=0 else 0 for x in for_cm_pred]

#print(classification_report(all_true_test, all_pred_test, target_names=test_tns))

In [ ]:
cm7 = confusion_matrix(all_true_test, all_pred_test, labels=test_tns_ids)
cmn7 = cm7.astype('float') / cm7.sum(axis=1)[:, np.newaxis]

In [ ]:
confusion7_np = cmn7
FP = confusion7_np.sum(axis=0) - np.diag(confusion7_np)
FN = confusion7_np.sum(axis=1) - np.diag(confusion7_np)
TP = np.diag(confusion7_np)
TN = confusion7_np.sum() - (FP + FN + TP)


# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP)
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
#F1-Score
F1 = TP/(TP+.5*(FP+FN))

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

metrics = {'FP':FP, 'FN':FN, 'TP':TP, 'TN':TN, 'TPR':TPR, 'TNR':TNR, 'PPV':PPV, 'NPV':NPV, 'FPR':FPR, 'FNR':FNR, 'FDR':FDR, 'F1':F1, 'ACC':ACC}
for k, v in metrics.items():
    print(k)
    for x in zip(test_tns, list(v)):
        print(x[0], x[1])
    print()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(cmn7, annot=True, fmt='.2f', xticklabels=cwes, yticklabels=cwes)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

plt.subplot(1, 2, 1)
sns1 = sns.heatmap(cmn7, annot=True, fmt='.2f', ax=ax1, xticklabels=cwes, yticklabels=cwes, annot_kws={"size": 15})
plt.ylabel('Actual', fontsize=15)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
cbar1 = sns1.collections[0].colorbar
cbar1.ax.tick_params(labelsize=14)


plt.subplot(1, 2, 2)
sns2 = sns.heatmap(cmn, annot=True, fmt='.2f', ax=ax2, xticklabels=labels_name_11, yticklabels=labels_name_11, annot_kws={"size": 14})
plt.ylabel('Actual', fontsize=15)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
cbar2 = sns2.collections[0].colorbar
cbar2.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig('Figure_A1.jpg', format='jpg', dpi=1200)
plt.show()